<a href="https://colab.research.google.com/github/aagamaar/Algo_Trading_Python/blob/main/AlgoPy_1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOUNTED GOOGLE DRIVE

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

Mounted at /content/drive
Google Drive mounted successfully!


CREATING PROJECT DIRECTORY STRUCTURE

In [3]:
# Creating Project Directory Structure
import os

# Defining the project root in Google Drive
project_root = '/content/drive/MyDrive/algo_trading_prototype'

# Creating the main project directory
os.makedirs(project_root, exist_ok=True)
print(f"Project root created: {project_root}")

# Creating subdirectories
subdirectories = [
    'config',
    'data',
    'strategy',
    'backtester',
    'analytics',
    'sheets',
    'utils',
    'logs' # Directory for log files
]

for subdir in subdirectories:
    path = os.path.join(project_root, subdir)
    os.makedirs(path, exist_ok=True)
    print(f"Created directory: {path}")

# I have added __init__.py files to make directories into Python packages
# This is crucial for Python to recognize the folders as modules for importing.

for pkg_dir in ['data', 'strategy', 'backtester', 'analytics', 'sheets', 'utils']:
    with open(os.path.join(project_root, pkg_dir, '__init__.py'), 'w') as f:
        pass # Created an empty __init__.py file
    print(f"Created __init__.py in {pkg_dir}")

print("\nDirectory structure created successfully.")

Project root created: /content/drive/MyDrive/algo_trading_prototype
Created directory: /content/drive/MyDrive/algo_trading_prototype/config
Created directory: /content/drive/MyDrive/algo_trading_prototype/data
Created directory: /content/drive/MyDrive/algo_trading_prototype/strategy
Created directory: /content/drive/MyDrive/algo_trading_prototype/backtester
Created directory: /content/drive/MyDrive/algo_trading_prototype/analytics
Created directory: /content/drive/MyDrive/algo_trading_prototype/sheets
Created directory: /content/drive/MyDrive/algo_trading_prototype/utils
Created directory: /content/drive/MyDrive/algo_trading_prototype/logs
Created __init__.py in data
Created __init__.py in strategy
Created __init__.py in backtester
Created __init__.py in analytics
Created __init__.py in sheets
Created __init__.py in utils

Directory structure created successfully.


INSTALLING THE DEPENDENCIES

In [4]:
# Installing the Dependencies
# Used --quiet to suppress verbose output
!pip install pandas numpy ta gspread oauth2client scikit-learn requests yfinance python-telegram-bot --quiet
print("All required Python dependencies installed.")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 22.9 MB/s eta 0:00:00
All required Python dependencies installed.


HOLDS ALL CONFIGURATIONS

In [6]:
# Cell 4: config/settings.py
%%writefile /content/drive/MyDrive/algo_trading_prototype/config/settings.py
# config/settings.py

# Stock Data API (using yfinance)
# No API key is needed for basic yfinance usage
STOCK_SYMBOLS = ["RELIANCE.NS", "TCS.NS", "HDFCBANK.NS"] # Three NIFTY 50 stocks for NSE.

# Google Sheets
GOOGLE_SHEET_ID = "1W8nP1H7oIKwdy7m5dGii0lZcf_cKxn0mPSdzhAddm80"
TRADE_LOG_SHEET_NAME = "Trade Log"
SUMMARY_PL_SHEET_NAME = "Summary P&L"
WIN_RATIO_SHEET_NAME = "Win Ratio"

# Strategy Parameters
RSI_PERIOD = 14
RSI_BUY_THRESHOLD = 30
SHORT_MA_PERIOD = 20 # 20-Day Moving Average
LONG_MA_PERIOD = 50  # 50-Day Moving Average

# Backtesting Parameters
BACKTEST_DURATION_MONTHS = 6

# ML Model Parameters
FEATURES = ['RSI', 'MACD', 'Volume', 'Close'] # Features for ML model
TARGET = 'Next_Day_Movement' # Targets for ML model

# Telegram Alerts (Bonus)
TELEGRAM_BOT_TOKEN = "8144019769:AAF-f7tW-XV9URIgJAAFyQgNtE0Tce0naXw"
TELEGRAM_CHAT_ID = "1463467106"

Overwriting /content/drive/MyDrive/algo_trading_prototype/config/settings.py


SETTING UP LOGGING TO BOTH CONSOLE AND A FILE IN LOGS DIRECTORY

In [9]:
%%writefile /content/drive/MyDrive/algo_trading_prototype/utils/logger.py
# utils/logger.py

import logging
import os
from datetime import datetime

def setup_logging():
    """Configures logging for the application."""
    # Adjusted log_dir to be within your mounted Google Drive project structure.
    log_dir = '/content/drive/MyDrive/algo_trading_prototype/logs'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    log_file = os.path.join(log_dir, f"algo_trading_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

    logging.basicConfig(
        level=logging.INFO, # Log INFO, WARNING, ERROR, CRITICAL messages
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file), # Log to a file
            logging.StreamHandler()        # Also print to console
        ]
    )
    # Suppressing verbose logging from libraries to keep console clean
    logging.getLogger('requests').setLevel(logging.WARNING)
    logging.getLogger('urllib3').setLevel(logging.WARNING)
    logging.getLogger('yfinance').setLevel(logging.WARNING)
    logging.getLogger('gspread').setLevel(logging.WARNING)

Overwriting /content/drive/MyDrive/algo_trading_prototype/utils/logger.py


HANDLES FETCHING STOCK DATA USING YFINANCE

In [11]:
%%writefile /content/drive/MyDrive/algo_trading_prototype/data/data_fetcher.py
# data/data_fetcher.py

import yfinance as yf
import pandas as pd
import logging
from datetime import datetime, timedelta
# Import BACKTEST_DURATION_MONTHS from settings for use in get_historical_data
from config import settings # Import settings module directly

logger = logging.getLogger(__name__)

class DataFetcher:
    def __init__(self):
        # yfinance doesn't require an API key for basic usage
        pass

    def fetch_historical_data(self, symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
        """
        Fetches historical stock data for a given symbol using yfinance.
        Parameters:
            symbol (str): Stock ticker symbol (e.g., "RELIANCE.NS").
            start_date (str): Start date in 'YYYY-MM-DD' format.
            end_date (str): End date in 'YYYY-MM-DD' format.
        Returns:
            pd.DataFrame: DataFrame with historical OHLCV data, or empty if fetching fails.
        """
        try:
            # yf.download returns a DataFrame directly with uppercase columns
            df = yf.download(symbol, start=start_date, end=end_date, progress=False) # progress=False to reduce console output during download
            if df.empty:
                logger.warning(f"No data fetched for {symbol} between {start_date} and {end_date}.")
                return pd.DataFrame()

            # Ensure we have the standard OHLCV columns and index is named 'Date'
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
            df.index.name = 'Date'
            logger.info(f"Successfully fetched historical data for {symbol}.")
            return df
        except Exception as e:
            logger.error(f"Error fetching data for {symbol} using yfinance: {e}")
            return pd.DataFrame()

def get_historical_data(symbols: list, duration_months: int) -> dict:
    """
    Fetches historical data for multiple symbols for the last 'duration_months' using yfinance.
    Parameters:
        symbols (list): List of stock ticker symbols.
        duration_months (int): Number of months for which to fetch historical data.
    Returns:
        dict: A dictionary where keys are symbols and values are pandas DataFrames.
    """
    data_fetcher = DataFetcher()
    all_data = {}
    end_date = datetime.now()
    # Calculate start date based on duration_months
    start_date = end_date - timedelta(days=duration_months * 30) # Approximate 30 days per month

    # Format dates as 'YYYY-MM-DD' for yfinance
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    for symbol in symbols:
        df = data_fetcher.fetch_historical_data(symbol, start_date_str, end_date_str)
        if not df.empty:
            all_data[symbol] = df
    return all_data

Overwriting /content/drive/MyDrive/algo_trading_prototype/data/data_fetcher.py
